In [61]:
import numpy as np
import pandas as pd
from patsy import dmatrices

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import datasets
from sklearn.linear_model import LinearRegression

In [45]:
usage = pd.read_csv('elec_usage.txt')
usage['yearcts'] = usage['year'] + (usage['month']-1)/12

## try several models:
  * 'usage ~ temperature + np.power(temperature, 2) + yearcts'

does using higher powers of temperature help?

In [111]:
y, X = dmatrices('usage ~ temperature + np.power(temperature, 2)+ np.power(temperature, 3) + np.power(temperature, 4) + yearcts', \
                 data=usage, return_type='dataframe')
lm = LinearRegression()
model = lm.fit(X,y)
model.score(X,y)

0.8485106210442108

In [112]:
X_train, X_test, y_train, y_test = train_test_split(
...     X, y, test_size=0.2, random_state=0)
X_train.shape, y_train.shape
X_test.shape, y_test.shape

((11, 6), (11, 1))

In [113]:
model = lm.fit(X_train, y_train)
print("Score on train set is", model.score(X_train, y_train))
print("Score on test set is", model.score(X_test, y_test))

Score on train set is 0.858237996215455
Score on test set is 0.7952141994787905


# cross validation
compute test score across several train / test splits

In [120]:
y, X = dmatrices('usage ~ temperature + np.power(temperature, 2) + yearcts', \
                 data=usage, return_type='dataframe')
lm = LinearRegression()
model = lm.fit(X,y)
np.mean(cross_val_score(model, X, y, cv=3))

0.7220203489706113

# Feature selection

## Select from model
use the features with largest coefficients according to some other model

In [ ]:
from sklearn.datasets import load_boston
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LassoCV

# Load the boston dataset.
X, y = load_boston(return_X_y=True)

# We use the base estimator LassoCV since the L1 norm promotes sparsity of features.
clf = LassoCV()

# Set a minimum threshold of 0.25
sfm = SelectFromModel(clf, threshold=0.25)
sfm.fit(X, y)
n_features = sfm.transform(X).shape[1]

# Reset the threshold till the number of features equals two.
# Note that the attribute can be set directly instead of repeatedly
# fitting the metatransformer.
while n_features > 2:
    sfm.threshold += 0.1
    X_transform = sfm.transform(X)
    n_features = X_transform.shape[1]

# Plot the selected two features from X.
plt.title(
    "Features selected from Boston using SelectFromModel with "
    "threshold %0.3f." % sfm.threshold)
feature1 = X_transform[:, 0]
feature2 = X_transform[:, 1]
plt.plot(feature1, feature2, 'r.')
plt.xlabel("Feature number 1")
plt.ylabel("Feature number 2")
plt.ylim([np.min(feature2), np.max(feature2)])
plt.show()